In [30]:
include("./src/structs.jl")
include("./src/scalar_operators.jl")
include("./src/broadcast_operators.jl")
include("./src/graph.jl")
include("./src/forward.jl")
include("./src/backward.jl")
include("./src/data_loader.jl")
include("./src/nn_model.jl")
include("./src/optimisers.jl")
include("./src/losses.jl")
include("./src/layers.jl")
include("./src/recurrent.jl")

using Printf, Random
Random.seed!(0)

TaskLocalRNG()

In [13]:
using JLD2
X_train = load("./data_rnn/imdb_dataset_prepared.jld2", "X_train")
y_train = load("./data_rnn/imdb_dataset_prepared.jld2", "y_train")
X_test = load("./data_rnn/imdb_dataset_prepared.jld2", "X_test")
y_test = load("./data_rnn/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("./data_rnn/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("./data_rnn/imdb_dataset_prepared.jld2", "vocab")
nothing

In [23]:
println("X_train: ", size(X_train))
println("y_train: ", size(y_train))
println("X_test: ", size(X_test))
println("y_test: ", size(y_test))
println("embeddings: ", size(embeddings))
println("vocab: ", size(vocab))

In [24]:
vocab_size = length(vocab)
embed_dim = size(embeddings, 1)  # 50 from your comment
sequence_length = size(X_train, 1)  # 130 from your comment
batch_size = 1

1

In [28]:
model = Chain(
    Embedding(vocab_size, embed_dim, name="embedding"),
    RNN(embed_dim, 16, return_sequences=false, name="rnn_layer"),
    Dense((16 => 1), σ, name="output_layer")
)

    
model.layers[1].weights.output .= embeddings


accuracy(y_true, y_pred) = mean((y_true .> 0.5) .== (y_pred .> 0.5))

accuracy (generic function with 1 method)

In [16]:
dataset = DataLoader((X_train, y_train), batchsize=batch_size, shuffle=true)
testset = DataLoader((X_test, y_test), batchsize=batch_size, shuffle=false)

DataLoader{Tuple{LinearAlgebra.Adjoint{Int64, Matrix{Int64}}, BitMatrix}, TaskLocalRNG}(([2909 1428 … 172 1841; 3067 9406 … 3324 1676; … ; 12849 3392 … 6695 286; 12849 2411 … 9521 286], Bool[0 1 … 0 0]), 10000, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000], 1, 10000, false, TaskLocalRNG())

In [31]:

net = NeuralNetwork(model, RMSProp(), binary_cross_entropy, accuracy)

MethodError: MethodError: no method matching size(::Nothing)
The function `size` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  size(!Matched::BitVector)
   @ Base bitarray.jl:104
  size(!Matched::BitVector, !Matched::Integer)
   @ Base bitarray.jl:107
  size(!Matched::Core.Compiler.StmtRange)
   @ Base show.jl:2820
  ...


In [ ]:
epochs = 12
for epoch in 1:epochs
    t = @elapsed begin
        train_loss, train_acc = train!(net, dataset)
    end
    
    test_loss, test_acc = evaluate(net, testset)
    @printf("Epoch %d/%d: Train Loss: %.4f, Train Acc: %.4f, Test Loss: %.4f, Test Acc: %.4f, Time: %.2fs\n",
            epoch, epochs, train_loss, train_acc, test_loss, test_acc, t)
end

Chain{Tuple{Embedding, RNN{typeof(ReLU)}, typeof(get_last), typeof(flatten), Dense{typeof(σ)}}}((Embedding(Variable(Float32[0.011746252 0.011771972 … 0.008536924 -0.005072477; 0.006740404 0.011394465 … 0.018728161 0.010205761; … ; 0.013949568 -0.0186823 … -0.0066614444 0.015276386; 0.0052829185 -0.016880201 … -0.012988941 0.007946664], nothing, "embedding")), RNN{typeof(ReLU)}(RNNCell{typeof(ReLU)}(Variable(Float32[0.084092565 -0.2840545 … 0.16767122 0.10863211; 0.2502509 0.2688337 … 0.17747103 -0.24526584; … ; -0.24021575 0.08683379 … 0.28555062 0.258427; -0.0031146007 -0.25705355 … -0.004985575 -0.26176253], nothing, "rnn_W_ih"), Variable(Float32[-0.41465202 -0.0959148 … -0.06520751 0.39138314; 0.27736914 -0.10750067 … -0.2300193 0.045505572; … ; -0.31230676 -0.17743583 … 0.087863095 -0.024300275; 0.22127126 0.3128932 … -0.13270226 -0.2242362], nothing, "rnn_W_hh"), Variable(Float32[0.0; 0.0; … ; 0.0; 0.0;;], nothing, "rnn_bias"), Main.ReLU, 16), true), Main.get_last, Main.flatten, D